In [73]:
import cv2
import numpy as np
import math

source = 'girl'

img = cv2.imread('./images/normalize_'+source+'.png')

#density = cv2.imread('./images/DistanceFromMean.png')

'''
if img.shape != density.shape:
    print("* * * two images have different shape:\n", "image: ", img.shape)
    print("density map: ", density.shape)
'''

height = img.shape[0]
width = img.shape[1]


kernel_right = np.array([[  1,  0, -1], 
                         [  2,  0, -2], 
                         [  1,  0, -1]])

kernel_top = np.array([[ -1, -2, -1], 
                       [  0,  0,  0], 
                       [  1,  2,  1]])

kernel_left = np.array([[ -1,  0,  1], 
                        [ -2,  0,  2], 
                        [ -1,  0,  1]])

kernel_down = np.array([[  1,  2,  1], 
                        [  0,  0,  0], 
                        [ -1, -2, -1]])

#right
conv_right_1 = cv2.filter2D(src=img, ddepth=-1, kernel=kernel_right)
conv_right_8 = cv2.resize(cv2.filter2D(src=cv2.resize(img, (width//8, height//8), interpolation=cv2.INTER_AREA), ddepth=-1, kernel=kernel_right), (width,height), interpolation=cv2.INTER_CUBIC) 
conv_right_8 //= 10
conv_right = cv2.add(conv_right_1, conv_right_8)

#top
conv_top_1 = cv2.filter2D(src=img, ddepth=-1, kernel=kernel_top)
conv_top_8 = cv2.resize(cv2.filter2D(src=cv2.resize(img, (width//8, height//8), interpolation=cv2.INTER_AREA), ddepth=-1, kernel=kernel_top), (width,height), interpolation=cv2.INTER_CUBIC) 
conv_top_8 //= 10
conv_top = cv2.add(conv_top_1, conv_top_8)

#left
conv_left_1 = cv2.filter2D(src=img, ddepth=-1, kernel=kernel_left)
conv_left_8 = cv2.resize(cv2.filter2D(src=cv2.resize(img, (width//8, height//8), interpolation=cv2.INTER_AREA), ddepth=-1, kernel=kernel_left), (width,height), interpolation=cv2.INTER_CUBIC) 
conv_left_8 //= 10
conv_left = cv2.add(conv_left_1, conv_left_8)

#down
conv_down_1 = cv2.filter2D(src=img, ddepth=-1, kernel=kernel_down)
conv_down_8 = cv2.resize(cv2.filter2D(src=cv2.resize(img, (width//8, height//8), interpolation=cv2.INTER_AREA), ddepth=-1, kernel=kernel_down), (width,height), interpolation=cv2.INTER_CUBIC) 
conv_down_8 //= 10
conv_down = cv2.add(conv_down_1, conv_down_8)



# eliminate high frequency noise
blur_size = 21
conv_right = cv2.GaussianBlur(conv_right, (blur_size, blur_size),0)
conv_top = cv2.GaussianBlur(conv_top, (blur_size, blur_size),0)
conv_left = cv2.GaussianBlur(conv_left, (blur_size, blur_size),0)
conv_down = cv2.GaussianBlur(conv_down, (blur_size, blur_size),0)


cv2.imwrite('conv_right.png', conv_right)
cv2.imwrite('conv_top.png', conv_top)
cv2.imwrite('conv_left.png', conv_left)
cv2.imwrite('conv_down.png', conv_down)





True

In [91]:
# combine 4 conv_out to get coarse light map
conv_right = cv2.imread('conv_right.png')
conv_top = cv2.imread('conv_top.png')
conv_left = cv2.imread('conv_left.png')
conv_down = cv2.imread('conv_down.png')

density = cv2.imread('./images/stroke_density_'+source+'.png')
density = cv2.imread('./images/cheat.png')
#density = cv2.imread('./images/'+source+'.png')
density = cv2.GaussianBlur(density, (3, 3),0)

original = cv2.imread('./images/'+source+'.png')
#mask = cv2.imread('./images/blind_mask.png')

# light
light_angle = 45+90# start from right side, clockwise
light_hue = 44   # 0~360
light_saturation = 200


light_direction = (light_angle/360) * 2*(math.pi)  # 0 ~ 2PI
#effectMap = conv_top * round((math.sin(light_direction))) + conv_right * round((math.cos(light_direction))) + (-1)*conv_down * (math.sin(light_direction)) + (-1)*conv_left * (math.cos(light_direction))
#lightMap = np.zeros_like(original)  # uint8
#lightMap = np.clip(effectMap.astype(np.uint8), 0, 255)
#shadowMap = np.clip(effectMap, -255, 0)

if light_angle%360 < 90:
    lightMap = conv_right * round(math.cos(light_direction)) + conv_top * round(math.sin(light_direction))
    shadowMap = conv_left * round(math.cos(light_direction)) + conv_down * round(math.sin(light_direction))
    
elif light_angle%360 < 180:
    lightMap = conv_top * round(math.cos(light_direction-math.pi/2)) + conv_left * round(math.sin(light_direction-math.pi/2))
    shadowMap = conv_down * round(math.cos(light_direction-math.pi/2)) + conv_right * round(math.sin(light_direction-math.pi/2))
    
elif light_angle%360 < 270:
    lightMap = conv_left * round(math.cos(light_direction-math.pi)) + conv_down * round(math.sin(light_direction-math.pi))
    shadowMap = conv_right * round(math.cos(light_direction-math.pi)) + conv_top * round(math.sin(light_direction-math.pi))

else:
    lightMap = conv_down * round(math.cos(light_direction-math.pi/2*3)) + conv_right * round(math.sin(light_direction-math.pi/2*3))
    shadowMap = conv_top * round(math.cos(light_direction-math.pi/2*3)) + conv_left * round(math.sin(light_direction-math.pi/2*3))

    
#lightMap = max(0, sign_1) * conv_right * round(math.cos(light_direction)) + max(0, sign_2) * conv_top * round(math.sin(light_direction)) + max(0, sign_1) * (-1) * conv_left * round(math.sin(light_direction)) + max(0, sign_2) * (-1) * conv_down * round(math.cos(light_direction))


hsv_lightMap = cv2.cvtColor(lightMap, cv2.COLOR_BGR2HSV)
hsv_lightMap[:,:,0] = light_hue // 2  # HSV in opencv, hue range is [0,179]
hsv_lightMap[:,:,1] = np.clip(hsv_lightMap[:,:,1], light_saturation, a_max=255)
#hsv_lightMap[:,:,2] = 0
lightMap = cv2.cvtColor(hsv_lightMap, cv2.COLOR_HSV2BGR)


hsv_shadowMap = cv2.cvtColor(shadowMap, cv2.COLOR_BGR2HSV)
#hsv_lightMap[:,:,0]
hsv_shadowMap[:,:,1] = 0
shadowMap = cv2.cvtColor(hsv_shadowMap, cv2.COLOR_HSV2BGR)


cv2.imwrite('lightMap.png', lightMap+127)
cv2.imwrite('shadowMap.png', shadowMap+127)


intensity = 5
result = original + lightMap * (1-(density/255)) * intensity - shadowMap * (1-(density/255)) * (intensity/1.5)# * (mask/255)
#reslut = result - shadowMap * (1-(density/255)) * 40 * (mask/255)


#cv2.imwrite('effectMap.png', effectMap+127)
cv2.imwrite('result.png', result)

all127 = np.clip(original, 127, 127)
all127 = all127 + lightMap* (1-(density/255))
all127 = all127 - shadowMap* (1-(density/255))
cv2.imwrite('effect.png', all127)


## Testing area
'''
lightMap += cv2.GaussianBlur(lightMap,(101,101),0)*1
lightMap += cv2.GaussianBlur(lightMap,(201,201),0)*1
lightMap += cv2.GaussianBlur(lightMap,(401,401),0)*1
'''
#effectMap = lightMap - shadowMap
#print(effectMap)
#cv2.imwrite('effectMap.png', effectMap+127)
#intensity = 1
#effectMap = effectMap * (1-(density/255)) * intensity #* (mask/255)
#result = original + effectMap





'\nlightMap += cv2.GaussianBlur(lightMap,(101,101),0)*1\nlightMap += cv2.GaussianBlur(lightMap,(201,201),0)*1\nlightMap += cv2.GaussianBlur(lightMap,(401,401),0)*1\n'

In [39]:
effectMap = cv2.imread('effectMap.png')
color = effectMap + original
cv2.imwrite('color.png', color)

True

1